In [102]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [3]:
!mkdir pdfs

In [5]:
loader=PyPDFDirectoryLoader('pdfs')

In [9]:
data=loader.load()

In [10]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [11]:
text_chunks=text_splitter.split_documents(data)

In [17]:
print(text_chunks[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto


In [18]:
print(text_chunks[1].page_content)

aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,


In [22]:
from dotenv import load_dotenv

In [46]:
load_dotenv()

True

In [26]:
key=os.getenv("OPENAI_API_KEY")

In [29]:
embedding=OpenAIEmbeddings()

In [47]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

In [72]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("test")

In [69]:
index_name="test"

In [70]:
embedding=OpenAIEmbeddings()

In [97]:
vector_store = PineconeVectorStore(index_name=index, embedding=embedding)
    
batch_size = 10
for i in range(0, len(text_chunks), batch_size):
    chunk_batch = text_chunks[i:i + batch_size]
    vector_store.add_documents(chunk_batch)

In [119]:
vector_store.similarity_search("What is self attention")

[Document(page_content='described in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a representation of the sequence. Self-attention has been\nused successfully in a variety of tasks including reading comprehension, abstractive summarization,\ntextual entailment and learning task-independent sentence representations [4, 27, 28, 22].', metadata={'page': 1.0, 'source': 'pdfs\\transformer.pdf'}),
 Document(page_content='during training.\n4 Why Self-Attention\nIn this section we compare various aspects of self-attention layers to the recurrent and convolu-\ntional layers commonly used for mapping one variable-length sequence of symbol representations\n(x1, ..., x n)to another sequence of equal length (z1, ..., z n), with xi, zi∈Rd, such as a hidden\nlayer in a typical sequence transduction encoder or decoder. Motivating our use of self-attention we\nconsider three desiderata.', 

In [95]:
#index.delete(delete_all=True)

{}

In [103]:
llm=OpenAI()

In [104]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

In [121]:
query="what are End-to-end memory networks based on?"

In [122]:
qa.invoke(query)

{'query': 'what are End-to-end memory networks based on?',
 'result': ' End-to-end memory networks are based on a recurrent attention mechanism.'}

In [108]:
vector_store.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000148E89115B0>)

In [123]:
!echo >>.gitignore